In [ ]:
from capymoa.ocl.evaluation import ocl_train_eval_loop
from capymoa.ocl.datasets import SplitMNIST
from capymoa.ann import Perceptron
from capymoa.classifier import Finetune
from capymoa.ocl.strategy import ExperienceReplay
from matplotlib import pyplot as plt
from torchvision.utils import make_grid
from torch.optim import Adam
import random

pt = 1 / 72.27
figsize_169 = (455 * pt, 256 * pt)
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = ["Noto Sans"]
plt.rcParams["font.size"] = 9

In [ ]:
scenario = SplitMNIST()
model = Perceptron(scenario.schema)
learner = ExperienceReplay(
    Finetune(
        scenario.schema,
        model,
        optimizer=Adam(model.parameters(), lr=0.001),
        device="cuda",
        batch_size=128,
    ),
    buffer_size=256,
)
results = ocl_train_eval_loop(
    learner,
    scenario.train_streams,
    scenario.test_streams,
    progress_bar=True,
    continual_evaluations=2,
)

In [ ]:
%load_ext autoreload
%autoreload 2
from plot import plot_metrics

fig, ax = plt.subplots()
plot_metrics(
    results,
    ax=ax,
    # online_acc=True,
    acc_all=True,
    acc_seen=True,
)

In [ ]:
results.accuracy_all_avg